## Hyperparameter Tuning

Hiperparametre, değeri öğrenme sürecini kontrol etmek için kullanılan bir parametredir.

Hiperparametre optimizasyonu veya tuning, bir öğrenme algoritması için bir dizi optimal hiperparametre seçme problemidir. Biz burada bunları yapmak için keras-tuner'ı kullanacağız...

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
import kerastuner
import matplotlib.pyplot as plt
import numpy

Veri setimiz keras'ın içinde hazır olarak bulunan Fashion Mnist veri setidir.

Alttaki işlemde eğitim ve test için veriler ayrılıyor...

In [ ]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.fashion_mnist.load_data()

Alttaki fonksiyonda ise eğer 'hp' ile fonksiyona bir şey gönderilmediyse normal ayarlarımızla modeli ayarlıyoruz.
Eğer gönderildiyse *if* ile parametrelerimizi belirliyoruz.

Burada *hp.Choice* ile değerlerimizin alabileceği değerleri girip rastgele seçmesini bekliyoruz.

*hp.Float* ile minimum ve maksimum değerlerini belirleyerek aradaki değerlerden seçmesini sağlıyoruz.

Geri kalan kısımda ise modelimizi oluşturuyoruz.

In [ ]:
def create_model(hp=None):
  num_hidden_layers=1
  num_units=8
  dropout_rate=0.1
  learning_rate=0.01
  if hp:
    num_hidden_layers=hp.Choice('num_hidden_layers',values=[1,2,3])
    num_units=hp.Choice('num_units',values=[8,16,32])
    dropout_rate=hp.Float('dropout_rate',min_value=0.1,max_value=0.5)
    learning_rate=hp.Float('learning_rate',min_value=0.0001,max_value=0.01)

  model=tf.keras.models.Sequential([
                                    tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Lambda(lambda x:x/255.)                          
  ])
  for _ in range(0,num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units,activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
  model.add(tf.keras.layers.Dense(10,activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  return model

Burada ise *BayesianOptimization* kullanarak kendimize bir tuner oluşturuyoruz.

In [ ]:
class CustomTuner(kerastuner.tuners.BayesianOptimization):
  def run_trial(self,trial,*args,**kwargs):
    kwargs['batch_size']=trial.hyperparameters.Int('batch_size',32,128,step=32)
    super(CustomTuner,self).run_trial(trial,*args,**kwargs)

Üstte oluşturduğumuz tuner'a parametrelerini giriyoruz.

In [ ]:
tuner=CustomTuner(
    create_model,
    objective='val_accuracy',
    max_trials=20,
    directory='logs',
    project_name='Fashion_mnist',
    overwrite=True
)

*Search_space_summary* ile yaptığımız ayarları gözlemleyebiliriz.

In [ ]:
tuner.search_space_summary()

*tuner.search* ile eğitim işlemini başlatıyoruz. Burada model kendine en iyi olacak parametre kombinasyonlarını deniyor ve en iyiyi seçiyor.

In [ ]:
tuner.search(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=5,verbose=False,
)

In [ ]:
tuner.results_summary(1)

En iyi modelimizi seçerek ayarlarına bakıyoruz.

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
model.summary()

Modelimizi eğitiyoruz ve daha önceki sonuçlarımızdan daha iyi sonuçlar almayı bekliyoruz...

In [ ]:
_=model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=20,batch_size=128,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3)]
)

In [ ]:
model.evaluate(x_test, y_test)